In [2]:
import pandas as pd

box_df = pd.read_csv("./box_df.csv")
box_df

,Unnamed: 0,image_id,category_id,area,bbox,iscrowd,id,x_min,y_min,height,width,x_max,y_max,ctr_x,ctr_y,ratio
0,0,0,0,257301.66,"[197.6, 193.7, 547.8, 469.7]",0,0,197.6,193.7,547.8,469.7,745.4,663.4,471.50,428.55,1.166276
1,1,1,3,10402.56,"[0.0, 407.4, 57.6, 180.6]",0,1,0.0,407.4,57.6,180.6,57.6,588.0,28.80,497.70,0.318937
2,2,1,7,26259.36,"[0.0, 455.6, 144.6, 181.6]",0,2,0.0,455.6,144.6,181.6,144.6,637.2,72.30,546.40,0.796256
3,3,1,4,69096.17,"[722.3, 313.4, 274.3, 251.9]",0,3,722.3,313.4,274.3,251.9,996.6,565.3,859.45,439.35,1.088924
4,4,1,5,24164.58,"[353.2, 671.0, 233.7, 103.4]",0,4,353.2,671.0,233.7,103.4,586.9,774.4,470.05,722.70,2.260155
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23139,23139,4882,5,768591.81,"[0.0, 116.2, 944.1, 814.1]",0,23139,0.0,116.2,944.1,814.1,944.1,930.3,472.05,523.25,1.159686
23140,23140,4882,7,57309.72,"[302.1, 439.3, 265.2, 216.1]",0,23140,302.1,439.3,265.2,216.1,567.3,655.4,434.70,547.35,1.227210
23141,23141,4882,0,1772.74,"[511.3, 451.1, 58.7, 30.2]",0,23141,511.3,451.1,58.7,30.2,570.0,481.3,540.65,466.20,1.943709
23142,23142,4882,1,53008.67,"[255.0, 421.4, 271.7, 195.1]",0,23142,255.0,421.4,271.7,195.1,526.7,616.5,390.85,518.95,1.392619


In [21]:
# background 삭제한 이미지 생성하기
import math
import matplotlib.pyplot as plt
index = 10
for index in box_df.image_id.unique():
    x_min = math.floor(min(box_df[box_df.image_id == index].x_min))
    y_min = math.floor(min(box_df[box_df.image_id == index].y_min))
    x_max = math.ceil(max(box_df[box_df.image_id == index].x_max))
    y_max = math.ceil(max(box_df[box_df.image_id == index].y_max))

    img = plt.imread(f"/opt/ml/detection/dataset/train/{str(index).zfill(4)}.jpg").copy()
    img = img.transpose(2,0,1)

    img[:,:,:x_min] = 0
    img[:,:y_min,:] = 0
    img[:,:,x_max:] = 0
    img[:,y_max:,:] = 0

    img = img.transpose(1,2,0)
    
    plt.imsave(f"/opt/ml/detection/dataset/train_background/{str(index).zfill(4)}.jpg", img)

In [1]:
import os

f1 = open("/opt/ml/detection/dataset/YOLO/cv_train_1_clean_pseudo.txt","at")

for idx in range(4871):
    path = f"/opt/ml/detection/dataset/test/{str(idx).zfill(4)}.jpg"
    if os.path.isfile(path):
        f1.write(f"{path}\n")
f1.close()

In [34]:
# yolov5 clean data에 따라 경로 생성하기

import os

f1 = open("/opt/ml/detection/dataset/YOLO/cv_train_1.txt","rt")
f2 = open("/opt/ml/detection/dataset/YOLO/cv_train_1_clean.txt","wt")

while True:
    line = f1.readline()
    if not line:
        break
    f2.write(line[:-15]+"train_clean"+line[-10:])

In [45]:
f1 = open("/opt/ml/detection/dataset/YOLO/cv_val_1.txt","rt")
f2 = open("/opt/ml/detection/dataset/YOLO/cv_val_1_clean.txt","wt")

while True:
    line = f1.readline()
    if not line:
        break
    f2.write(line[:-15]+"train_clean"+line[-10:])
f1.close()
f2.close()

In [50]:
f1 = open("/opt/ml/detection/yolov5/convert2Yolo/manifest.txt","rt")
f2 = open("/opt/ml/detection/dataset/YOLO/cv_val_5_clean.txt","wt")

while True:
    line = f1.readline()
    if not line:
        break
    f2.write(line[:-15]+line[-9:])
f1.close()
f2.close()

In [2]:
# yolox datasets 폴더 만들기
import json
import pandas as pd

with open('/opt/ml/detection/dataset/cv2_train_5.json', 'r') as f:
    json_data = json.load(f)
    
image_df = pd.DataFrame(json_data['images'])

In [3]:
import shutil

for image_file in image_df.file_name:
    shutil.copy(f"/opt/ml/detection/dataset/train_clean/{image_file[-8:]}","/opt/ml/detection/YOLOX/datasets/COCO5/train2017")

In [4]:
with open('/opt/ml/detection/dataset/cv2_val_5.json', 'r') as f:
    json_data = json.load(f)
    
image_df = pd.DataFrame(json_data['images'])

for image_file in image_df.file_name:
    shutil.copy(f"/opt/ml/detection/dataset/train_clean/{image_file[-8:]}","/opt/ml/detection/YOLOX/datasets/COCO5/val2017")

In [ ]:
# from ipywidgets import interact
import ipywidgets
import matplotlib.pyplot as plt
import cv2

image_id = 3201
def show_object(image_id):
    img_root = f"/opt/ml/detection/dataset/train/{str(image_id).zfill(4)}.jpg"
    img = plt.imread(img_root)
    img_bboxes = box_df[box_df.image_id == image_id]
    for i in img_bboxes.index:
        x1 = int(img_bboxes['x_min'][i])
        y1 = int(img_bboxes['y_min'][i])
        x2 = int(img_bboxes['x_max'][i])
        y2 = int(img_bboxes['y_max'][i])
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)

    plt.figure(figsize=(15,15))
    plt.imshow(img)
def show_image(image_id):
    img_root = f"/opt/ml/detection/dataset/train/{str(image_id).zfill(4)}.jpg"
    img = plt.imread(img_root)
    plt.figure(figsize=(10,10))
    plt.imshow(img)

'''
interact(
    show_object,
    image_id = (3201, 4000)
)
'''

show_object (4193)

In [ ]:
def show_object(2971):
    img_root = f"/opt/ml/detection/dataset/train/{str(image_id).zfill(4)}.jpg"
    img = plt.imread(img_root)
    img_bboxes = box_df[box_df.image_id == image_id]
    for i in img_bboxes.index:
        x1 = int(img_bboxes['x_min'][i])
        y1 = int(img_bboxes['y_min'][i])
        x2 = int(img_bboxes['x_max'][i])
        y2 = int(img_bboxes['y_max'][i])
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)

    plt.figure(figsize=(15,15))
    plt.imshow(img)

In [8]:
# oversampling
with open('/opt/ml/detection/dataset/cv2_train_5.json', 'r') as f:
    json_data = json.load(f)
    
annot_df = pd.DataFrame(json_data['annotations'])
annot_df

,image_id,category_id,area,bbox,iscrowd,id
0,0,0,257301.6600,"[197.6, 193.7, 547.8, 469.7]",0,0
1,1,3,10402.5600,"[0.0, 407.4, 57.6, 180.6]",0,1
2,1,7,26259.3600,"[0.0, 455.6, 144.6, 181.6]",0,2
3,1,4,69096.1700,"[722.3, 313.4, 274.3, 251.9]",0,3
4,1,5,24164.5800,"[353.2, 671.0, 233.7, 103.4]",0,4
...,...,...,...,...,...,...
18330,4193,7,117112.0566,"[81, 590, 303.58, 385.77]",0,23157
18331,4210,0,10153.2288,"[601, 423, 122.98, 82.56]",0,23158
18332,4210,5,34613.4222,"[68, 649, 222.18, 155.79]",0,23159
18333,4237,0,26646.9630,"[260, 209, 374.15, 71.22]",0,23160


In [15]:
image_category = annot_df.groupby(["image_id","category_id"]).count()
image_category

area  bbox  iscrowd  id
image_id category_id                         
0        0               1     1        1   1
1        0               2     2        2   2
         3               1     1        1   1
         4               1     1        1   1
         5               2     2        2   2
...                    ...   ...      ...  ..
4879     5               1     1        1   1
         7               2     2        2   2
4880     0               1     1        1   1
4881     1               1     1        1   1
         7               3     3        3   3

[7568 rows x 4 columns]

In [16]:
image_sum = annot_df.groupby(["image_id"]).count().area
image_sum

image_id
0        1
1        8
3        2
4        6
5       22
        ..
4877     2
4878     1
4879     4
4880     1
4881     4
Name: area, Length: 3907, dtype: int64

In [31]:
# image_category['image_id'] = 
index = list(image_category.index)
image_id = [x[0] for x in index]
category_id = [x[1] for x in index]

In [32]:
image_category['image_id'] = image_id
image_category['category_id'] = category_id
image_category

area  bbox  iscrowd  id  image_id  category_id
image_id category_id                                                
0        0               1     1        1   1         0            0
1        0               2     2        2   2         1            0
         3               1     1        1   1         1            3
         4               1     1        1   1         1            4
         5               2     2        2   2         1            5
...                    ...   ...      ...  ..       ...          ...
4879     5               1     1        1   1      4879            5
         7               2     2        2   2      4879            7
4880     0               1     1        1   1      4880            0
4881     1               1     1        1   1      4881            1
         7               3     3        3   3      4881            7

[7568 rows x 6 columns]

In [34]:
image_category['sum_image'] = image_category['image_id'].apply(lambda x : image_sum[x])

In [35]:
image_category

area  bbox  iscrowd  id  image_id  category_id  \
image_id category_id                                                   
0        0               1     1        1   1         0            0   
1        0               2     2        2   2         1            0   
         3               1     1        1   1         1            3   
         4               1     1        1   1         1            4   
         5               2     2        2   2         1            5   
...                    ...   ...      ...  ..       ...          ...   
4879     5               1     1        1   1      4879            5   
         7               2     2        2   2      4879            7   
4880     0               1     1        1   1      4880            0   
4881     1               1     1        1   1      4881            1   
         7               3     3        3   3      4881            7   

                      sum_image  
image_id category_id             
0        0                    1  
1        0                    8  
         3                    8  
         4                    8  
         5                    8  
...                         ...  
4879     5                    4  
         7                    4  
4880     0                    1  
4881     1                    4  
         7                    4  

[7568 rows x 7 columns]

In [38]:
image_category['ratio'] = image_category['area'] / image_category.sum_image

In [39]:
image_category

area  bbox  iscrowd  id  image_id  category_id  \
image_id category_id                                                   
0        0               1     1        1   1         0            0   
1        0               2     2        2   2         1            0   
         3               1     1        1   1         1            3   
         4               1     1        1   1         1            4   
         5               2     2        2   2         1            5   
...                    ...   ...      ...  ..       ...          ...   
4879     5               1     1        1   1      4879            5   
         7               2     2        2   2      4879            7   
4880     0               1     1        1   1      4880            0   
4881     1               1     1        1   1      4881            1   
         7               3     3        3   3      4881            7   

                      sum_image  rati0  ratio  
image_id category_id                           
0        0                    1  1.000  1.000  
1        0                    8  0.250  0.250  
         3                    8  0.125  0.125  
         4                    8  0.125  0.125  
         5                    8  0.250  0.250  
...                         ...    ...    ...  
4879     5                    4  0.250  0.250  
         7                    4  0.500  0.500  
4880     0                    1  1.000  1.000  
4881     1                    4  0.250  0.250  
         7                    4  0.750  0.750  

[7568 rows x 9 columns]

In [80]:
len(image_category[(image_category['ratio'] >= 0.5) & (image_category.category_id == 0)].image_id.unique())

876

In [73]:
image_category[image_category.image_id ==]


,,area,bbox,iscrowd,id,image_id,category_id,sum_image,rati0,ratio
image_id,category_id,,,,,,,,,
4084,0,1,1,1,1,4084,0,1,1.0,1.0


In [67]:
image_category[(image_category['ratio'] > 0.7) & (image_category.category_id == 0)]

,,area,bbox,iscrowd,id,image_id,category_id,sum_image,rati0,ratio
image_id,category_id,,,,,,,,,
0,0,1,1,1,1,0,0,1,1.0,1.0
10,0,1,1,1,1,10,0,1,1.0,1.0
11,0,1,1,1,1,11,0,1,1.0,1.0
23,0,1,1,1,1,23,0,1,1.0,1.0
24,0,1,1,1,1,24,0,1,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...
4851,0,1,1,1,1,4851,0,1,1.0,1.0
4860,0,1,1,1,1,4860,0,1,1.0,1.0
4873,0,1,1,1,1,4873,0,1,1.0,1.0
